In [ ]:
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import confusion_matrix, classification_report

import matplotlib.pyplot as plt

In [ ]:
IMAGE_SIZE = 28

In [ ]:
x, y = fetch_openml('mnist_784', return_X_y=True)
y = y.astype(np.int32)

In [ ]:
#@title Code to add noise to MNIST images
class Deformation(object):

  def __init__(self, proba):
    self.proba = proba

  def transform(self, x):
    pass

  def __call__(self, x):
    if np.random.random() > self.proba:
      return x
    
    return self.transform(x)

class WhitePatch(Deformation):
  def transform(self, x):
    patch_size = np.random.randint(5, high=15, size=2)
    patch_location = np.random.randint(IMAGE_SIZE/5, IMAGE_SIZE - IMAGE_SIZE/5, size=2)
    x1, y1 = patch_location
    x2, y2 = np.clip(patch_size + patch_location, 0, IMAGE_SIZE - 1)
    x[y1: y2, x1: x2] = 255.
    return x

class DropPixels(Deformation):
  def transform(self, x):
    pix_2_drop = np.random.choice([0, 1], p=[.75, .25], size=(28, 28))
    pix_2_drop = np.where(pix_2_drop)
    x[pix_2_drop] = 255.
    return x

class DeformationPipeline(object):
  def __init__(self, *deformations):
    self.deformations = deformations
  
  def __call__(self, x):
    x = x.reshape(IMAGE_SIZE, IMAGE_SIZE)
    for d in self.deformations:
      x = d(x)
    return x.reshape(-1)

noise_mnist = DeformationPipeline(WhitePatch(7.), 
                                  DropPixels(8.))

In [ ]:
# Convert add noise to all data
noised_x = np.stack([noise_mnist(o) for o in x])

In [ ]:
cols = 10
rows = 1
plt.figure(figsize=(10, 10))
for l in range(10): # Digits from 0 to 9
  plt.subplot(rows, cols, l + 1)
  l_digits = noised_x[y == l]
  idx = np.random.randint(0, high=l_digits.shape[0], size=1)
  
  im = l_digits[idx]
  im = im.astype(np.float32) / 255.
  im = im.reshape(28, 28)
  
  plt.title(l)
  plt.imshow(im, cmap='gray')
  plt.axis('off')

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.33)

In [ ]:
np.savez_compressed('noised-MNIST.npz', x=x_train, y=y_train, x_submission=x_test)
np.savez_compressed('noised-MNIST-secret.npz', x=x_test, y=y_test)

In [ ]:
#@title Training code
class ImageScaler(BaseEstimator, TransformerMixin):

  def __init__(self, scale_factor):
    self.scale_factor = scale_factor

  def fit(self, x, y=None):
    return self
  
  def transform(self, x):
    x = x.astype(np.float64)
    x = x / float(self.scale_factor)
    x = np.clip(x, 0, 1.)
    return x

preprocessing_pipeline = Pipeline([('scale', ImageScaler(255)), 
                                   ('standardize', StandardScaler())])

x_train_prep = preprocessing_pipeline.fit_transform(x_train)
rnd_forest = RandomForestClassifier(n_estimators=20, max_depth=10)
rnd_forest.fit(x_train_prep, y_train)

In [ ]:
y_pred = rnd_forest.predict(preprocessing_pipeline.transform(x_test))